# Tratar Dados

# Bibliotecas

In [1]:
!pip install unidecode

In [2]:
import pandas as pd
from unidecode import unidecode

# Tratamento de dados dicionário

## Importar Dicionários

In [3]:
def abrir_dics(path, sheet_name='dicionário pnad covid'):

    # Lê arquivo Excel no caminho especificado (path)
    prepdics = pd.read_excel(path, sheet_name=sheet_name, header=1, usecols='B:F')

    # Remove a coluna 'Quesito' e as primeiras 3 linhas
    prepdics = prepdics.iloc[3:].drop('Quesito', axis=1)

    # Renomeia as colunas
    prepdics.columns = ['variavel', 'questao', 'tipo', 'descricao']

    # Reset do índice
    prepdics = prepdics.reset_index(drop=True)

    return prepdics

dic09 = abrir_dics('/content/drive/MyDrive/PosTech_Analise_de_dados/FASE3/tech_challenge_3/dados/Dicionario_PNAD_COVID_092020_20220621.xls')
dic10 = abrir_dics('/content/drive/MyDrive/PosTech_Analise_de_dados/FASE3/tech_challenge_3/dados/Dicionario_PNAD_COVID_102020_20220621.xls')
dic11 = abrir_dics('/content/drive/MyDrive/PosTech_Analise_de_dados/FASE3/tech_challenge_3/dados/Dicionario_PNAD_COVID_112020_20220621.xls')

dics = [dic09, dic10, dic11]

Visualizar primeiras alterações feitas, dos 3 dicionarios

In [4]:
print("dic09:")
print(dic09.head(5))
print(dic09.shape)

print("\ndic10:")
print(dic10.head(5))
print(dic10.shape)

print("\ndic11:")
print(dic11.head(5))
print(dic11.shape)

dic09:
  variavel               questao tipo descricao
0       UF  Unidade da Federação   11  Rondônia
1      NaN                   NaN   12      Acre
2      NaN                   NaN   13  Amazonas
3      NaN                   NaN   14   Roraima
4      NaN                   NaN   15      Pará
(641, 4)

dic10:
  variavel               questao tipo descricao
0       UF  Unidade da Federação   11  Rondônia
1      NaN                   NaN   12      Acre
2      NaN                   NaN   13  Amazonas
3      NaN                   NaN   14   Roraima
4      NaN                   NaN   15      Pará
(641, 4)

dic11:
  variavel               questao tipo descricao
0       UF  Unidade da Federação   11  Rondônia
1      NaN                   NaN   12      Acre
2      NaN                   NaN   13  Amazonas
3      NaN                   NaN   14   Roraima
4      NaN                   NaN   15      Pará
(656, 4)


## Tirar valores null e preencher campos

In [5]:
def limpar_dicionarios(*dfs):
    # Lista para armazenar os dataframes limpos
    clean_dfs = []
    for df in dfs:
        # Preenche os valores NaN e faz uma cópia
        clean = df.ffill().copy()
        # Converte para minúsculas as colunas de string, exceto a coluna 'tipo'
        clean = clean.apply(lambda x: x.str.lower() if x.name != 'tipo' and x.name != 'variavel' and x.dtype == 'object' else x)
        # Remove acentos e caracteres especiais
        clean = clean.applymap(lambda x: unidecode(str(x)) if isinstance(x, str) else x)
        # Substitui padrões nas colunas de string
        replaces = {'  ': ' ', r'\n': ' ', r'\r': ' ', r'/': ' ou ', ' - ': '_', '-': '_', r'%': '_por_cento'}
        clean.replace(replaces, regex=True, inplace=True)
        # Adiciona o dataframe limpo à lista
        clean_dfs.append(clean)
    return clean_dfs

clean_dfs = limpar_dicionarios(dic09, dic10, dic11)
dic09_clean, dic10_clean, dic11_clean = clean_dfs

dic_clean = [dic09_clean, dic10_clean, dic11_clean]


In [6]:
print("dic09:")
print(dic09_clean.head(5))
print(dic09_clean.shape)

print("\ndic10:")
print(dic10_clean.head(5))
print(dic10_clean.shape)

print("\ndic11:")
print(dic11_clean.head(5))
print(dic11_clean.shape)

dic09:
  variavel               questao tipo descricao
0       UF  unidade da federacao   11  rondonia
1       UF  unidade da federacao   12      acre
2       UF  unidade da federacao   13  amazonas
3       UF  unidade da federacao   14   roraima
4       UF  unidade da federacao   15      para
(641, 4)

dic10:
  variavel               questao tipo descricao
0       UF  unidade da federacao   11  rondonia
1       UF  unidade da federacao   12      acre
2       UF  unidade da federacao   13  amazonas
3       UF  unidade da federacao   14   roraima
4       UF  unidade da federacao   15      para
(641, 4)

dic11:
  variavel               questao tipo descricao
0       UF  unidade da federacao   11  rondonia
1       UF  unidade da federacao   12      acre
2       UF  unidade da federacao   13  amazonas
3       UF  unidade da federacao   14   roraima
4       UF  unidade da federacao   15      para
(656, 4)


## Concatena os dicionários

In [7]:
def processar_dicionarios(dic_clean, drop_duplicates=True):
    # Concatena todos os dicionários
    df_dics = pd.concat(dic_clean)
    # Preenche os valores NaN
    df_dics = df_dics.ffill()
    # Remove duplicatas
    if drop_duplicates:
        df_dics = df_dics.drop_duplicates()
    # Remove duplicatas com base em 'variavel' e 'descricao'
    df_dics = df_dics.drop_duplicates(subset=['variavel', 'descricao'])

    return df_dics

# Usando a função
df_dics_processed = processar_dicionarios(dic_clean, drop_duplicates=True)
print(df_dics_processed.head(5), "\nshape:", df_dics_processed.shape)

  variavel               questao tipo descricao
0       UF  unidade da federacao   11  rondonia
1       UF  unidade da federacao   12      acre
2       UF  unidade da federacao   13  amazonas
3       UF  unidade da federacao   14   roraima
4       UF  unidade da federacao   15      para 
shape: (650, 4)


## Visualizar as questões

In [8]:
def questions (df, variavel=None, key='tipo', value='questao'):
    if variavel is None:
        return df.set_index(key)[value].to_dict()
    else:
        return df.query(f'variavel == "{variavel}"').set_index(key)[value].to_dict()

# Visualizar todas as questões
questionsdic = questions(df_dics_processed, key='variavel', value='questao')
questionsdic

{'UF': 'unidade da federacao',
 'CAPITAL': 'capital',
 'RM_RIDE': 'regiao metropolitana e regiao administrativa integrada de desenvolvimento',
 'V1008': 'numero de selecao do domicilio',
 'V1012': 'semana no mes',
 'V1013': 'mes da pesquisa',
 'V1016': 'numero da entrevista no domicilio',
 'Estrato': 'estrato',
 'UPA': 'upa',
 'V1022': 'situacao do domicilio',
 'V1023': 'tipo de area',
 'V1030': 'projecao da populacao',
 'V1031': 'peso do domicilio e das pessoas',
 'V1032': 'peso do domicilio e das pessoas',
 'posest': 'dominios de projecao',
 'A001': 'numero de ordem',
 'A001A': 'condicao no domicilio',
 'A001B1': 'dia de nascimento',
 'A001B2': 'mes de nascimento',
 'A001B3': 'ano de nascimento',
 'A002': 'idade do morador ',
 'A003': 'sexo',
 'A004': 'cor ou raca',
 'A005': 'escolaridade',
 'A006': 'frequenta escola',
 'A007': 'na semana passada, _____ foram disponibilizadas atividades escolares para realizar em casa?',
 'A008': 'na semana passada, em quantos dias _____ dedicou_se a

## Criar excel do dicionário

In [9]:
df_dics_processed.to_excel("/content/drive/MyDrive/PosTech_Analise_de_dados/FASE3/tech_challenge_3/dados/dicionario_tratado.xlsx", index = False)

# Tratamento de dados PNAD COVID

3 meses para construção da solução

In [10]:
df09 = pd.read_csv('/content/drive/MyDrive/PosTech_Analise_de_dados/FASE3/tech_challenge_3/dados/PNAD_COVID_092020.csv')
df10 = pd.read_csv('/content/drive/MyDrive/PosTech_Analise_de_dados/FASE3/tech_challenge_3/dados/PNAD_COVID_102020.csv')
df11 = pd.read_csv('/content/drive/MyDrive/PosTech_Analise_de_dados/FASE3/tech_challenge_3/dados/PNAD_COVID_112020.csv')

covid_dfs = [df09, df10, df11]

for d in covid_dfs:
    print(f'shape: {d.shape}')

shape: (387298, 145)
shape: (380461, 145)
shape: (381438, 148)


## Compara as colunas

In [11]:
def comparar_colunas(df1, df2):
    cols1 = set(df1.columns)
    cols2 = set(df2.columns)

    cols_only_in_df1 = cols1 - cols2
    cols_only_in_df2 = cols2 - cols1

    if cols_only_in_df1:
        print(f"Colunas presentes apenas em df1: {cols_only_in_df1}")
    if cols_only_in_df2:
        print(f"Colunas presentes apenas em df2: {cols_only_in_df2}")

# Comparar todos os pares de DataFrames
comparar_par = [(df09, df10), (df09, df11), (df10, df11)]

for df1, df2 in comparar_par:
    comparar_colunas(df1, df2)

Colunas presentes apenas em df2: {'A006A', 'A007A', 'A006B'}
Colunas presentes apenas em df2: {'A006A', 'A007A', 'A006B'}


## Concatenar

In [12]:
df_concat = pd.concat([df09, df10, df11])
df_concat.shape

(1149197, 148)

## Selecionar as colunas

In [13]:
# Lista de variáveis selecionadas
variaveis_selecionadas = ['UF' ,'A002' ,'A003' ,'A004' ,'A005' ,'A006A' ,'A006' ,'F001' ,'V1022' ,'C01011' ,'B011' ,'B0011' ,'B0012'
                          ,'B0013' ,'B0014' ,'B0015' ,'B0016' ,'B0017' ,'B0018' ,'B0019' ,'B00110' ,'B00111' ,'B00112' ,'B00113' ,'B002'
                          ,'B0043' ,'B0046' ,'B0041' ,'B0044' ,'B0042' ,'B0045' ,'B008' ,'B009A' ,'B009B' ,'B009C' ,'B009D' ,'B009E'
                          ,'B009F' ,'B0101' ,'B0102' ,'B0103' ,'B0104' ,'B0105' ,'B0106' ,'C002' ,'C003' ,'C004' ,'C006' ,'C007'
                          ,'C008' ,'C013']

# Selecionando apenas as colunas desejadas
df_covid = df_concat[variaveis_selecionadas]

## Ajustado os dados usando do dicionário

In [14]:
def rename_codes(df, var_names):
    for var_name in var_names:
        # Crie um dicionário de códigos e descrições
        code_dict = df_dics_processed.query(f'variavel == "{var_name}"')[['tipo', 'descricao']].set_index('tipo')['descricao'].to_dict()

        # Renomeie a variável no DataFrame
        df.loc[:, var_name] = df[var_name].map(code_dict)

    return df


# Lista de variáveis para renomear
var_names_to_rename = ['UF', 'A003', 'A004', 'F001', 'V1022','B0011', 'B0012', 'B0013',
                       'B0014','B0015','B0016', 'B0017','B0018', 'B0019', 'B00110',
                       'B00111', 'B00112', 'B00113', 'B008','B0101','B0102', 'B0103',
                       'B0104', 'B0105', 'B0106']


# Renomeie as variáveis no DataFrame
df_covid = rename_codes(df_covid, var_names_to_rename)

# Exiba as primeiras linhas do DataFrame
df_covid.head()

,UF,A002,A003,A004,A005,A006A,A006,F001,V1022,C01011,...,B0104,B0105,B0106,C002,C003,C004,C006,C007,C008,C013
0,rondonia,36,homem,parda,5,NaN,NaN,proprio_ja pago,urbana,4.0,...,nao,nao,nao,NaN,NaN,NaN,2.0,4.0,48.0,NaN
1,rondonia,30,mulher,parda,7,NaN,NaN,proprio_ja pago,urbana,4.0,...,nao,nao,nao,NaN,NaN,NaN,2.0,7.0,36.0,NaN
2,rondonia,13,homem,parda,2,NaN,1.0,proprio_ja pago,urbana,NaN,...,nao,nao,nao,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,rondonia,11,homem,parda,2,NaN,1.0,proprio_ja pago,urbana,NaN,...,nao,nao,nao,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,rondonia,57,mulher,branca,2,NaN,NaN,proprio_ja pago,urbana,NaN,...,nao,nao,nao,2.0,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
columns_to_map = {
    'A006A': {1.0: 'pública', 2.0: 'privada'},
    'A006':{1.0: 'sim', 2.0: 'nao'},
    'C002': {1.0: 'sim', 2.0: 'nao'},
    'C006': {1.0: 'sim', 2.0: 'nao'},
    'C013': {1.0: 'sim', 2.0: 'nao'},
    'C004': {1.0: 'sim', 2.0: 'nao', 3.0: 'nao remunerado'},
    'B002': {1.0: 'sim', 2.0: 'nao', 9.0: 'nao aplicavel'},
    'B0041': {1.0: 'sim', 2.0: 'nao', 9.0: 'nao aplicavel'},
    'B0042': {1.0: 'sim', 2.0: 'nao', 9.0: 'nao aplicavel'},
    'B0043': {1.0: 'sim', 2.0: 'nao', 9.0: 'nao aplicavel'},
    'B0044': {1.0: 'sim', 2.0: 'nao', 9.0: 'nao aplicavel'},
    'B0045': {1.0: 'sim', 2.0: 'nao', 9.0: 'nao aplicavel'},
    'B0046': {1.0: 'sim', 2.0: 'nao', 9.0: 'nao aplicavel'},
    'B009A': {1.0: 'sim', 2.0: 'nao', 9.0: 'nao aplicavel'},
    'B009C': {1.0: 'sim', 2.0: 'nao', 9.0: 'nao aplicavel'},
    'B009E': {1.0: 'sim', 2.0: 'nao', 9.0: 'nao aplicavel'},
    'B009B': {1.0: 'positivo', 2.0: 'negativo', 3.0: 'inconclusivo', 4.0: 'nao recebeu o resultado', 9.0: 'nao aplicavel'},
    'B009D': {1.0: 'positivo', 2.0: 'negativo', 3.0: 'inconclusivo', 4.0: 'nao recebeu o resultado', 9.0: 'nao aplicavel'},
    'B009F': {1.0: 'positivo', 2.0: 'negativo', 3.0: 'inconclusivo', 4.0: 'nao recebeu o resultado', 9.0: 'nao aplicavel'}
}

for col, mapping in columns_to_map.items():
    df_covid[col] = df_covid[col].map(mapping).fillna('nao aplicavel')

<ipython-input-15-0356e08ea7a4>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_covid[col] = df_covid[col].map(mapping).fillna('nao aplicavel')


In [16]:
# Mapeamento para a coluna C003 - tipo de trabalho
a005_mapping = {
    1:  'sem instrucao',
    2:  'fundamental incompleto',
    3:  'fundamental completa',
    4:  'medio incompleto',
    5:  'medio completo',
    6:  'superior incompleto',
    7:  'superior completo',
    8:  'pos mestrado doutorado',
}

# Mapear a coluna C003
df_covid['A005'] = df_covid['A005'].map(a005_mapping).fillna('nao aplicavel')

<ipython-input-16-d5168a44c52b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_covid['A005'] = df_covid['A005'].map(a005_mapping).fillna('nao aplicavel')


In [17]:
# Mapeamento para a coluna C003 - tipo de trabalho
c003_mapping = {
    1.0:  'quarentena ou ferias coletivas',
    2.0:  'ferias ou jornada variavel',
    3.0:  'maternidade ou paternidade',
    4.0:  'saude ou acidente remunerada',
    5.0:  'outro tipo de licenca remunerada',
    6.0:  'afastamento sem ser remunerado',
    7.0:  'fatores ocasionais',
    8.0:  'outro motivo',
}

# Mapear a coluna C003
df_covid['C003'] = df_covid['C003'].map(c003_mapping).fillna('nao aplicavel')

<ipython-input-17-2b1ffd24f38f>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_covid['C003'] = df_covid['C003'].map(c003_mapping).fillna('nao aplicavel')


In [18]:
# Mapeamento para a coluna C01011
c01011_mapping = {
    0.0: '0 a 100',
    1.0: '101 a 300',
    2.0: '301 a 600',
    3.0: '601 a 800',
    4.0: '801 a 1.600',
    5.0: '1.601 a 3.000',
    6.0: '3.001 a 10.000',
    7.0: '10.001 a 50.000',
    8.0: '50.001 a 100.000',
    9.0: 'mais de 100.000'
}

# Mapear a coluna C01011
df_covid['C01011'] = df_covid['C01011'].map(c01011_mapping).fillna('nao aplicavel')

<ipython-input-18-864adab6d5a4>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_covid['C01011'] = df_covid['C01011'].map(c01011_mapping).fillna('nao aplicavel')


In [19]:
# Mapeamento para a coluna C007 - tipo de trabalho
c007_mapping = {
    1.0: 'trabalhador domestico',
    2.0: 'militar',
    3.0: 'policial ou bombeiro militar',
    4.0: 'empregado do setor privado',
    5.0: 'empregado do setor público',
    6.0: 'empregador',
    7.0: 'conta propria',
    8.0: 'trabalhador familiar',
    9.0: 'fora do mercado de trabalho'
}

# Mapear a coluna C007
df_covid['C007'] = df_covid['C007'].map(c007_mapping).fillna('nao aplicavel')

<ipython-input-19-198b631b9ba7>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_covid['C007'] = df_covid['C007'].map(c007_mapping).fillna('nao aplicavel')


In [20]:
# Mapeamento para a coluna B011 - restringiu_contato_fisico
b011_mapping = {
    1: 'sem restricao',
    2: 'reduziu o contato',
    3: 'so saiu para necessidade basica',
    4: 'ficou somente em casa',
    9: 'nao aplicavel'
}

# Mapear a coluna B011
df_covid['B011'] = df_covid['B011'].map(b011_mapping).fillna('nao aplicavel')

<ipython-input-20-208dc9fa3e7c>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_covid['B011'] = df_covid['B011'].map(b011_mapping).fillna('nao aplicavel')


In [21]:
def add_region_column(df):
  regions = {
      'norte': ['acre', 'amazonas', 'roraima', 'amapa', 'para', 'tocantins', 'rondonia'],
      'nordeste': ['maranhao', 'piaui', 'ceara', 'rio grande do norte', 'paraiba', 'pernambuco', 'alagoas', 'sergipe', 'bahia'],
      'sudeste': ['minas gerais', 'espirito santo', 'rio de janeiro', 'sao paulo'],
      'sul': ['parana', 'santa catarina', 'rio grande do sul'],
      'centro-oeste': ['mato grosso', 'mato grosso do sul', 'goias', 'distrito federal']
  }
  df['regiao'] = df['UF'].apply(lambda uf: next((region for region, states in regions.items() if uf in states), 'nao identificado'))
  return df

df_covid = add_region_column(df_covid)
df_covid.head()

<ipython-input-21-c9ba88667adc>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['regiao'] = df['UF'].apply(lambda uf: next((region for region, states in regions.items() if uf in states), 'nao identificado'))


,UF,A002,A003,A004,A005,A006A,A006,F001,V1022,C01011,...,B0105,B0106,C002,C003,C004,C006,C007,C008,C013,regiao
0,rondonia,36,homem,parda,medio completo,nao aplicavel,nao aplicavel,proprio_ja pago,urbana,801 a 1.600,...,nao,nao,nao aplicavel,nao aplicavel,nao aplicavel,nao,empregado do setor privado,48.0,nao aplicavel,norte
1,rondonia,30,mulher,parda,superior completo,nao aplicavel,nao aplicavel,proprio_ja pago,urbana,801 a 1.600,...,nao,nao,nao aplicavel,nao aplicavel,nao aplicavel,nao,conta propria,36.0,nao aplicavel,norte
2,rondonia,13,homem,parda,fundamental incompleto,nao aplicavel,sim,proprio_ja pago,urbana,nao aplicavel,...,nao,nao,nao aplicavel,nao aplicavel,nao aplicavel,nao aplicavel,nao aplicavel,NaN,nao aplicavel,norte
3,rondonia,11,homem,parda,fundamental incompleto,nao aplicavel,sim,proprio_ja pago,urbana,nao aplicavel,...,nao,nao,nao aplicavel,nao aplicavel,nao aplicavel,nao aplicavel,nao aplicavel,NaN,nao aplicavel,norte
4,rondonia,57,mulher,branca,fundamental incompleto,nao aplicavel,nao aplicavel,proprio_ja pago,urbana,nao aplicavel,...,nao,nao,nao,nao aplicavel,nao aplicavel,nao aplicavel,nao aplicavel,NaN,nao aplicavel,norte


## Verificar null

In [22]:
df_covid.isnull().sum()

UF             0
A002           0
A003           0
A004           0
A005           0
A006A          0
A006           0
F001           0
V1022          0
C01011         0
B011           0
B0011          0
B0012          0
B0013          0
B0014          0
B0015          0
B0016          0
B0017          0
B0018          0
B0019          0
B00110         0
B00111         0
B00112         0
B00113         0
B002           0
B0043          0
B0046          0
B0041          0
B0044          0
B0042          0
B0045          0
B008           0
B009A          0
B009B          0
B009C          0
B009D          0
B009E          0
B009F          0
B0101          0
B0102          0
B0103          0
B0104          0
B0105          0
B0106          0
C002           0
C003           0
C004           0
C006           0
C007           0
C008      713200
C013           0
regiao         0
dtype: int64

## Preencher os NaN

In [23]:
# NaN to 0
df_covid = df_covid.fillna(0)
df_covid.head()

,UF,A002,A003,A004,A005,A006A,A006,F001,V1022,C01011,...,B0105,B0106,C002,C003,C004,C006,C007,C008,C013,regiao
0,rondonia,36,homem,parda,medio completo,nao aplicavel,nao aplicavel,proprio_ja pago,urbana,801 a 1.600,...,nao,nao,nao aplicavel,nao aplicavel,nao aplicavel,nao,empregado do setor privado,48.0,nao aplicavel,norte
1,rondonia,30,mulher,parda,superior completo,nao aplicavel,nao aplicavel,proprio_ja pago,urbana,801 a 1.600,...,nao,nao,nao aplicavel,nao aplicavel,nao aplicavel,nao,conta propria,36.0,nao aplicavel,norte
2,rondonia,13,homem,parda,fundamental incompleto,nao aplicavel,sim,proprio_ja pago,urbana,nao aplicavel,...,nao,nao,nao aplicavel,nao aplicavel,nao aplicavel,nao aplicavel,nao aplicavel,0.0,nao aplicavel,norte
3,rondonia,11,homem,parda,fundamental incompleto,nao aplicavel,sim,proprio_ja pago,urbana,nao aplicavel,...,nao,nao,nao aplicavel,nao aplicavel,nao aplicavel,nao aplicavel,nao aplicavel,0.0,nao aplicavel,norte
4,rondonia,57,mulher,branca,fundamental incompleto,nao aplicavel,nao aplicavel,proprio_ja pago,urbana,nao aplicavel,...,nao,nao,nao,nao aplicavel,nao aplicavel,nao aplicavel,nao aplicavel,0.0,nao aplicavel,norte


In [24]:
df_covid.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1149197 entries, 0 to 381437
Data columns (total 52 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   UF      1149197 non-null  object 
 1   A002    1149197 non-null  int64  
 2   A003    1149197 non-null  object 
 3   A004    1149197 non-null  object 
 4   A005    1149197 non-null  object 
 5   A006A   1149197 non-null  object 
 6   A006    1149197 non-null  object 
 7   F001    1149197 non-null  object 
 8   V1022   1149197 non-null  object 
 9   C01011  1149197 non-null  object 
 10  B011    1149197 non-null  object 
 11  B0011   1149197 non-null  object 
 12  B0012   1149197 non-null  object 
 13  B0013   1149197 non-null  object 
 14  B0014   1149197 non-null  object 
 15  B0015   1149197 non-null  object 
 16  B0016   1149197 non-null  object 
 17  B0017   1149197 non-null  object 
 18  B0018   1149197 non-null  object 
 19  B0019   1149197 non-null  object 
 20  B00110  1149197 non-null  obje

## Renomear as colunas

In [25]:
# Dicionário com os novos nomes das variáveis
df_covid = df_covid.rename(columns={
    # Social
    'UF': 'uf',
    'A002': 'idade',
    'A003': 'sexo',
    'A004': 'cor',
    'A005': 'escolaridade',
    'A006A': 'ensino_publico_privado',
    'A006': 'frequentou_escola',
    'F001': 'tipo_moradia',
    'V1022': 'situacao_domiciliar',
    #Renda
    'C01011': 'rendimento',
    # Comportamento (lockdown)
    'B011': 'restringiu_contato_fisico',
    # Sintomas
    'B0011': 'febre',
    'B0012': 'tosse',
    'B0013': 'dor_garganta',
    'B0014': 'dificuldade_respirar',
    'B0015': 'dor_cabeca',
    'B0016': 'dor_peito',
    'B0017': 'nausea',
    'B0018': 'nariz_entupido',
    'B0019': 'fadiga',
    'B00110': 'dor_olhos',
    'B00111': 'perda_olfato_paladar',
    'B00112': 'dor_muscular',
    'B00113': 'diarreia',
    # Medico (Social)
    'B002':  'ida_unidade_saude',
    'B0043': 'hospital_publico',
    'B0046': 'hospital_privado',
    'B0041': 'posto_publico',
    'B0044': 'ambulatorio_privado',
    'B0042': 'pronto_socorro_publico',
    'B0045': 'pronto_socorro_privado',
    # Testes (covid)
    'B008': 'fez_teste',
    'B009A': 'teste_swab', # exame coletado com cotonete na boca e ou ou nariz
    'B009B': 'resultado_swab',
    'B009C': 'teste_sangue_dedo',
    'B009D': 'resultado_sangue_dedo',
    'B009E': 'teste_sangue_coleta',
    'B009F': 'resultado_sangue_coleta',
    # Comorbidade
    'B0101': 'diagnostico_diabetes',
    'B0102': 'diagnostico_hipertensao',
    'B0103': 'diagnostico_doenca_pulmao',
    'B0104': 'diagnostico_doenca_cardiaca',
    'B0105': 'diagnostico_depressao',
    'B0106': 'diagnostico_cancer',
    #Afastamento (trabalho)
    'C002': 'afastado_trabalho',
    'C003': 'motivo_do_afastamento',
    'C004': 'remunerado_continua',
    # trabalho
    'C006': 'mais_de_um_trabalho',
    'C007': 'tipo_trabalho',
    'C008': 'horas_trabalho_normal',
    'C013': 'trabalho_remoto_semana_passada'
})

df_covid.head()

,uf,idade,sexo,cor,escolaridade,ensino_publico_privado,frequentou_escola,tipo_moradia,situacao_domiciliar,rendimento,...,diagnostico_depressao,diagnostico_cancer,afastado_trabalho,motivo_do_afastamento,remunerado_continua,mais_de_um_trabalho,tipo_trabalho,horas_trabalho_normal,trabalho_remoto_semana_passada,regiao
0,rondonia,36,homem,parda,medio completo,nao aplicavel,nao aplicavel,proprio_ja pago,urbana,801 a 1.600,...,nao,nao,nao aplicavel,nao aplicavel,nao aplicavel,nao,empregado do setor privado,48.0,nao aplicavel,norte
1,rondonia,30,mulher,parda,superior completo,nao aplicavel,nao aplicavel,proprio_ja pago,urbana,801 a 1.600,...,nao,nao,nao aplicavel,nao aplicavel,nao aplicavel,nao,conta propria,36.0,nao aplicavel,norte
2,rondonia,13,homem,parda,fundamental incompleto,nao aplicavel,sim,proprio_ja pago,urbana,nao aplicavel,...,nao,nao,nao aplicavel,nao aplicavel,nao aplicavel,nao aplicavel,nao aplicavel,0.0,nao aplicavel,norte
3,rondonia,11,homem,parda,fundamental incompleto,nao aplicavel,sim,proprio_ja pago,urbana,nao aplicavel,...,nao,nao,nao aplicavel,nao aplicavel,nao aplicavel,nao aplicavel,nao aplicavel,0.0,nao aplicavel,norte
4,rondonia,57,mulher,branca,fundamental incompleto,nao aplicavel,nao aplicavel,proprio_ja pago,urbana,nao aplicavel,...,nao,nao,nao,nao aplicavel,nao aplicavel,nao aplicavel,nao aplicavel,0.0,nao aplicavel,norte


## Exporta dados tratados

In [26]:
sintomas = df_covid[['tosse', 'dor_garganta', 'dificuldade_respirar', 'dor_cabeca', 'dor_peito', 'nausea', 'fadiga', 'dor_olhos', 'perda_olfato_paladar', 'dor_muscular', 'diarreia']]
sintomas.head()

,tosse,dor_garganta,dificuldade_respirar,dor_cabeca,dor_peito,nausea,fadiga,dor_olhos,perda_olfato_paladar,dor_muscular,diarreia
0,nao,nao,nao,nao,nao,nao,nao,nao,nao,nao,nao
1,nao,nao,nao,nao,nao,nao,nao,nao,nao,nao,nao
2,nao,nao,nao,nao,nao,nao,nao,nao,nao,nao,nao
3,nao,nao,nao,nao,nao,nao,nao,nao,nao,nao,nao
4,nao,nao,nao,nao,nao,nao,nao,nao,nao,nao,nao


In [27]:
df_covid.to_csv('/content/drive/MyDrive/PosTech_Analise_de_dados/FASE3/tech_challenge_3/dados/df_covid.csv', index = False)